In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "cubert_varmisuse_tiny"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [4]:
unpersist(subgraph_partition)

,id,train_mask,val_mask,test_mask
0,5040,True,False,False
1,9406,True,False,False
2,9720,True,False,False
3,5332,True,False,False
4,6923,True,False,False
...,...,...,...,...
23087,993090,False,False,True
23088,998594,False,False,True
23089,998064,False,False,True
23090,997465,False,False,True


In [5]:
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in
0,0,subword,483174,425064,5040,NaN
1,1,arg,425064,97124,5040,142323.0
2,2,arg_rev,97124,425064,5040,142323.0
3,3,args,97124,248751,5040,142323.0
4,4,args_rev,248751,97124,5040,142323.0
5,5,args,248751,142323,5040,142323.0
6,6,args_rev,142323,248751,5040,142323.0
7,7,subword,786198,143287,5040,NaN
8,8,decorator_list,143287,142323,5040,142323.0
9,9,decorator_list_rev,142323,143287,5040,142323.0


In [6]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
)

In [7]:
save_config(config, "var_misuse_tiny.yaml")

In [8]:
config

{'DATASET': {'data_path': 'cubert_varmisuse_tiny',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': 'cubert_varmisuse_tiny\\partition.json.bz2'},
 'TRAINING': {'model_output_dir': 'cubert_varmisuse_tiny',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 100,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': False,
  'use_ns_groups': Fals

# Create Dataset

In [9]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [10]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [11]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SubgraphClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [12]:
%tensorboard --logdir data_path

In [ ]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 1014088


Epoch 1:   0%|                                                                                 | 0/110 [00:00<?, ?it/s]

Epoch: 0, Time: 342 s
{'Accuracy/test/VariableMisuseClf_': 0.5029166666666667,
 'Accuracy/train/VariableMisuseClf_': 0.5,
 'Accuracy/train_avg/VariableMisuseClf': 0.5012073863636364,
 'Accuracy/val/VariableMisuseClf_': 0.4907852564102564,
 'Loss/test/VariableMisuseClf_': 0.6933931122223537,
 'Loss/train/VariableMisuseClf_': 0.6923571825027466,
 'Loss/train_avg/VariableMisuseClf': 0.694703086939725,
 'Loss/val/VariableMisuseClf_': 0.6938766489426295}


Epoch 1:  63%|█████████████████████████████████████████████▏                          | 69/110 [02:27<02:09,  3.17s/it]